In [1]:
from psbasis import * # Loading our package

%display latex
n = PSBasis.n(None)

$\def\cc{\mathbb{C}}\def\nn{\mathbb{N}}\def\kk{\mathbb{K}}\def\qq{\mathbb{Q}}\def\zz{\mathbb{Z}}$
$\def\basis{\mathfrak{B}}\def\ibasis{\mathfrak{I}}\def\pbasis{\mathfrak{P}}\def\bbasis{\mathfrak{C}}\def\hbasis{\mathfrak{H}}$
# Inverse Zeibelger Problem

In this notebook we implement Petkovšek's algorithm DefiniteSumsSolutions in Sage using the basis ''ore_algebra'' package by M. Kauers and M. Mezzaroba.

The main idea of this algorithm is translating recurrence equations that has no simple solution to a new difference equation which we may can compute a solution after assuming some hypergeometric terms in the original solution.

### Input and output

The algorithm receives a P-finite equation $L$ and some numbers $a_1,...,a_m \in \nn$ and some elements $b_1,...,b_m \in \kk$.

The algorithm returns another P-finite equation $L'$, such that $(h_n)_{n\geq 0}$ is a solution of $L'\cdot h = 0$ if and only if
$$L\cdot\left(\sum_{k\geq0}\prod_{i=1}^m\binom{a_in+b_i}{k}h_k\right) = 0$$

## 1. Compatible basis

The idea of this algorithm is to change the usual representation of a sequence stablishing a different basis. Usually, for power series $\kk[[x]]$, we use the _power basis_ $x^n$. However, there are other basis for the power series that, sometimes, help to write and solve a prolem from a new angle.

Let $\basis = \{P_n\ :\ n\in\nn\}$ be a basis over $\kk$ of the ring of formal power series $\kk[[x]]$. Although Petkovšek is mainly interested in basis formed by polynomials, we are going to include here also other type of power series basis.

* $\basis$ is said to be a **polynomial basis** if $\deg(P_n) = n$.
* $\basis$ is said to be an **order basis** if $ord_x(P_n) = x$.
* $\basis$ is **$(A,B)$-compatible** for a linear operator $L$ if for any $n \in \nn$
  $$L\cdot P_n = \sum_{k = -A}^{B} \alpha_{n,k}P_{n+k},$$
  i.e., applying the linear operator $L$ to $P_n$ expands the basis with a finite bound around $P_n$.
  
Petkovšek is also interested in a particular type of polynomial basis that will prove to be useful for the following results. Here we take the chance to add the definition for orthogonal polynomials.

* $\basis$ is a **factorial basis** if for all $n \in \nn$, $P_n | P_{n+1}$. This means, there is $p_n(x) \in \kk[x]$ such that $P_{n+1}(x) = P_n(x)p_n(x)$.
  - *Remark*: in the particular case of polynomial basis, $p_n(x) = a_nx + b_n$ for some sequences $(a_n)$ and $(b_n)$.
* $\basis$ is a **orthogonal basis** if it is a *polynomial basis* such that for all $n \in \nn$ there are $a_n$, $b_n$ and $c_n$ in $\kk$ such that
  $$P_{n+1} = (a_nx + b_n)P_n - c_nP_{n-1}.$$


### Example of compatible basis

In this section we will see how the usual polynomial basis are compatible with some basic linear operators. We will follow the notation:

* $\pbasis$ will denote the _power basis_ where $P_n = x^n$.
* $\pbasis_{a,b}$ will denote the _general power basis_ where $P_n = (ax + b)^n$.
* $\bbasis$ will denote the _binomial basis_ where $P_n = \binom{x}{n}$.
* $\bbasis_{a,b}$ will denote the _general binomial basis_ where $P_n = \binom{ax+b}{n}$.
* $\hbasis$ will denote the basis of orthogonal _Hermite polynomial_ $P_n = H_n$

And for the operators we will use:

1. $D$ for standard derivation in $\kk[[x]]$.
2. $E$ for the _shift_ operator $x \rightarrow (x+1)$.
3. $Q$ for the _q-shift_ operator $x \rightarrow (qx)$.
4. $X$ for the _multiplication by $x$_ operator.

Then it is clear by definition that:
* Every polynomial factorial basis is $(0,1)$-compatible with $X$, since $P_n(a_nx + b_n) = P_{n+1}$, so:
  $$xP_n = \frac{1}{a_n}\left(P_{n+1} - b_nP_n\right).$$
* Every orthogonal basis is $(-1,1)$-compatible with $X$ using the three terms recurrence:
  $$xP_n = \frac{1}{a_n}\left(c_nP_{n-1} - b_nP_n + P_{n+1}\right)$$
* $\pbasis_{a,b}$ is $(1,0)$-compatible with $D$:
  $$((ax+b)^n)' = an(ax+b)^{n-1}.$$
* $\pbasis$ is $(0,0)$-compatible with $Q$:
  $$Q(x^n) = q^n(x^n).$$
* $\pbasis$ is not compatible with $E$: since $(x+1)^n = \sum_{k=0}^n\binom{n}{k}x^k$.
* $\bbasis$ is $(1,0)$-compatible with $E$:
  $$\binom{x+1}{n} = \binom{x}{n} + \binom{x}{n-1}.$$
* $\bbasis_{a,b}$ is $(a,0)$-compatible with $E$ (as it is shown in Proposition 6 of Petkovšek's paper):
  $$\binom{a(x+1)+b}{n} = \sum_{i=-a}^0\binom{a}{-i} \binom{ax+b}{n+i}.$$
* $\hbasis$ is $(1,0)$-compatible with $D$ since:
  $$H_n'(x) = 2nH_{n-1}(x).$$
  
##### What is it implemented?

In the package `psbasis`, we have implemented this system of basis and compatibility. The following classes are available:
* `PSBasis`: represents any _power series basis_. This class is abstract.
* `PolyBasis`: represents any _polynomial series basis_. This class is abstract.
* `FactorialBasis`: represents any _factorial basis_. This class is abstract.
* `SFactorialBasis`: a simple implementation of `FactorialBasis`. It is created specifying the sequences $a_n$ and $b_n$ such that $P_{n} = (a_nx + b_n)P_{n+1}$. Using these sequence the compatibility with $X$ is automatically computed.
* `OrthogonalBasis`: represents any _orthogonal basis_. It is created specifying the sequences $a_n$, $b_n$ and $c_n$ of the three term recurrence. Using these sequence the compatibility with $X$ is automatically computed.

Moreover, several of the examples are implemented in different classes to simplify the input:
* `PowerBasis`: represents $\pbasis_{a,b}$. These constants can be specified when creating the object or the the usual $\bbasis$ with $a=1$ and $b=0$ is given. It includes the compatibility with $D$.
* `BinomialBasis`: represents $\bbasis_{a,b}$. These constants can be specified when creating the object or the the usual $\bbasis$ with $a=1$ and $b=0$ is given. It includes automatically the compatibility with $E$.
* `HermiteBasis`: represents $\hbasis$. It includes the compatibility with $D$.
 

In [2]:
P = PowerBasis(); x = P.polynomial_ring().gens()[0]
print(all(P.element(i) == x^i for i in range(20)))
P._PSBasis__compatibility

True


{'x': (0,
  1,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63af5d30>),
 'Id': (0,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63ac1b80>),
 'Dx': (1,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63ac1d30>)}

The method ``compatibility(L)`` receives the operator $L$ (either its name or the object in an ``OreAlgebra`` and returns the  compatibility description of the basis with such operator. The output has the following format $(A,B,m,\alpha_{i,j}(n))$, where:

* $A$ is the lower bound of the compatibility condition. See also the method ``lower_bound`` or simply ``A``.
* $B$ is the upper bound of the compatibility condition. See also the method ``upper_bound`` or simply ``B``.
* $m$ is the number of sections in which the compatibility is divided.
* $\alpha_{i,j}(n)$ are the coefficients of the compatibility such that, for all $n = km + r$:

  $$L \cdot P_n = \sum_{j=-A}^B \alpha_{r,j}(k)P_{n+i}.$$
  See also the method the method ``compatibility_coefficient`` or simply ``alpha``.
  
The format for the $\alpha_{i,j}(n)$ is a method with the three arguments $(i,j,n)$. This can be easier visualize with the method ``compatibility_matrix``, which returns the matrix $\left(\alpha_{i,j-A}(n)\right)_{i=0,...,m-1}^{j=0,...,A+B}$ together with the values of $A$ and $B$.

In [3]:
P.compatibility_matrix('Dx')

(1, 0, [n 0])

In [4]:
P11 = PowerBasis(1,1)
print([P11.element(i).factor() for i in range(5)])
P11._PSBasis__compatibility

[1, x + 1, (x + 1)^2, (x + 1)^3, (x + 1)^4]


{'x': (0,
  1,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63ac1820>),
 'Id': (0,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63af2280>),
 'Dx': (1,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63af2430>)}

In [5]:
B = BinomialBasis()
print(all(B.element(i) == binomial(x,i) for i in range(20)))
B._PSBasis__compatibility

True


{'x': (0,
  1,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63af21f0>),
 'E': (1,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed63af2160>)}

In [6]:
H = HermiteBasis();
print(all(H.element(i) == hermite(i,x) for i in range(20)))
H._PSBasis__compatibility

True


{'x': (1,
  1,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed6297ba60>),
 'Dx': (1,
  0,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed6297bc10>)}

### Scalar equivalence

One basic operation for series of the power series is the scalar multiplication. Let $\basis$ be a basis of $\kk[[x]]$ with general term $P_n(x)$ and $(c_n)_{n\in \kk}$ be a sequence of constants such that $c_n \neq 0$. Then the sequence $(c_nP_n(x))_{n\in \kk}$ is also a basis of $\kk[[x]]$. Moreover:

**Lemma:** let $\basis = (P_n(x))_n$ be a basis of $\kk[[x]]$, $(c_n)_n \in \kk^\nn$ and $L$ a $(A,B)$-compabitle operator with $\basis$. Let $Q_n(x) = c_nP_n(x)$ and $\basis' = (Q_n(x))_n$. Then:
* If $\basis$ is factorial, then $\basis'$ is factorial.
* If $\basis$ is an order basis, then $\basis'$ is an order basis.
* $L$ is $(A,B)$-compatible with $\basis'$.

The first two points need no proof, since $c_n \in \kk \setminus \{0\}$, the order and degree are preserved. Moreover, if $P_n(x)$ are polynomials, multiplying by $c_n$ does not change the roots, hence if $\basis$ is factorial, so is $\basis'$.

Now, assume that the compatibility condition of $L$ with $\basis$ can be written as:
$$L(P_n(x)) = \sum_{i=-A}^B \alpha_{n,i}P_{n+i}(x),$$
then, using the fact that $L$ is $\kk$-linear:
$$L(c_nP_n(x)) = \sum_{i=-A}^B (\alpha_{n,i}\frac{c_n}{c_{n+i}} (c_{n+i}P_{n+i}(x)).$$

##### How is it implemented?

In order to obtain a suitable quotient $c_{n}/c_{n+i}$ for all $n$, we require that the sequence $(c_n)_n$ is hypergeometric. This means that these quotients are always rational functions in $i$ and $n$, and allows us to nively represent them into the system.

The class ``PSBasis`` defines a generic method ``scalar`` for scalar multiplication that first checks that the input is, indeed, hypergeometric. Depending if the sequence is a rational function or hypergeometric, this method will call either the method
``_scalar_basis`` or ``_scalar_hypergeometric``. These functions are specific for each class of the series implemented and return the structure for the new basis.

After that, the method ``scalar`` adds all the compatibilities that were computed already for the original basis.

We also allow the use of magic python syntax ``*`` with elements that can be casted into rational functions in $n$.

In [7]:
monic_B = B.scalar(factorial(n)); show(monic_B)
monic_B._PSBasis__compatibility

Factorial basis: (1, x, x^2 - x, ...)

{'x': (0,
  1,
  1,
  <function PSBasis.set_compatibility.<locals>.<lambda> at 0x7fed62925af0>),
 'E': (1,
  0,
  1,
  <function PSBasis.__scalar_hyper_extend_compatibilities.<locals>.<lambda> at 0x7fed6292d0d0>)}

In [8]:
(monic_B/factorial(n))[:10] == B[:10]

True

## 2. From compatible operators to recurrence equations

In Proposition 4, Petkovšek proved that, givel a basis $\basis = \{P_n\ :\ n \in \nn\}$ and a linear operator $L$ which is $(A,B)$-compatible, then it is equivalent, for $y(x) = \sum_{n\geq0}c_nP_n$:
* $L\cdot y = 0$.
* For all $n \geq 0$:
  $$\sum_{i=-B}^A \alpha_{n+i,-i}c_{n+i} = 0,$$
  where $c_k = 0$ whenever $k < 0$.
  
This leads to a recurrence equation for the $c_n$:
$$(R_{\basis}L)\cdot (c_n) = \left(\sum_{i=-B}^A \alpha_{n+i,-i}S_n^i\right)(c_n) = 0.$$
Hence, whenever the $\alpha_{n,i}$ are rational functions in $n$, we arrive to a new P-finite recurrence equation.

Moreover, in Theorem 1, Petkovšek proved that the map from $L$ to $R_{\basis}L$ is a ring homomorphism, which alow to simple computations of the recurrence operators. Given $L_1, L_2$ compatible with $\basis$ and $c \in \kk$, then we have:
* $R_{\basis}(cL_1) = cR_{\basis}(L_1)$.
* $R_{\basis}(L_1+L_2) = R_{\basis}(L_1) + R_{\basis}(L_2)$.
* $R_{\basis}(L_1L_2) = R_{\basis}(L_1)R_{\basis}(L_2)$.

This means that, if we have $L \in \kk[O_1,\dots,O_t]$ where $O_i$ are compatible operators with $\basis$, then to compute $R_{\basis}(L)$ we only need to evaluate the polynomial that represents $L$:
    $$L = Q(O_1,\dots,O_t) \Rightarrow R_{\basis}(L) = Q(R_{\basis}(O_1),\dots,R_{\basis}(O_t).$$

### What is it implemented?

In our package `psbasis` we have implemented this isomorphism with several methods:
* `recurrence(L)`: this method computes $R_{\basis}(L)$. This method requires that $L$ is expressed as a polynomial of operators that have been already set. 

The user can specified further compatibilities by giving the appropriate name for it and the corresponding value of $R_{\basis}(\cdot)$ or the values for $(A,B,m,\alpha_{i,j}(n))$ described previously.
* `set_compatibilty(L, R)`: set the compatibility of $L$ to $R$.

***REMARK***: the method ``set_compatibility`` only transforms recurrence to compatibilities assuming the value of $m$ is 1 (i.e., only 1 section is allowed). This is because, when we have multiple sections, the map from compatibilities to recurrence is not a isomorphism.

In [9]:
## Creating the operators we want to see their compatibility
OE.<E> = OreAlgebra(QQ[x], ('E', lambda p : p(x=x+1), lambda p : 0));

## Recall B is the binomial basis
B

Binomial basis (x) choose n

##### Example 4 from Petkovšek's paper

* At example 4.1, Petkovšek considers the operator $L = E - c$, so $y(x) = \sum_{n \geq 0} y_n \binom{x}{n}$ satisfies $y(x+1) - c = 0$ if and only if $L' \cdot (y_n) = 0$ where $L'$ is the changed operator to the binomial basis:

In [10]:
example4_1 = E - 3; B.recurrence(example4_1)

Sn - 2

  In general, $L' = S_n - (c-1)$, which in the sequence level has the solution $y_n = y_0(c-1)^n$, obtaining:
  $$y(x) = \sum_{n \geq 0} \binom{x}{n}y_0(c-1)^n = y_0c^x$$
  
* At example 4.2, Petkovšek considers the operator $L = E^2 - 2E + 1$. Then, applying the substitution for a binomial basis we get:

In [11]:
example4_2 = E^2 - 2*E + 1; B.recurrence(example4_2)

Sn^2

What does this result mean? Let $y(x) = \sum_{n\geq 0} y_n\binom{x}{n}$. Then we have that $y_{n+2} = 0$ for all $n$, so $y(x) = y_0 + y_1\binom{x}{1} = y_0 + y_1x$ and, then, all the power series solutions to the difference equation
$$y(x+2) - 2y(x+1) + y(x) = 0,$$
are the polynomials of degree 1.

* At example 4.3, Petkovšek now consider the operator $L = E^2 - E - 1$. Applying the algorithm we get:

In [12]:
example4_3 = E^2 - E - 1; B.recurrence(example4_3)

Sn^2 + Sn - 1

In this case we are considering a Fibonacci type of function, some $y(x)$ such that $y(x+2) = y(x+1) + y(x)$. If we expand it using the binomial basis, we end up that the sequence of coefficients satisfies the recurrence
$$y_{n+2} = y_n - y_{n+1},$$
which is a variance of the Fibonacci sequence. In fact, we have:
$$y_n = (-1)^n(C_1F_n + C_2F_{n+1}),$$
where $F_n$ **is** the Fibonacci sequence $(0,1,1,2,\dots)$. Let choose $C_1$ and $C_2$ such that $y(0) = 0$ and $y(1) = 1$. Then we have that $0 = y_0 = C_2$ and $1 = y_0 + y_1 = -(C_1 + C_2)$, so $C_1 = -1$ and $C_2 = 0$. Then $y_n = (-1)^nF_n$ and we have:
$$y(x) = \sum_{n \geq 0} (-1)^nF_n,$$
so evaluating the function on an integer $m$ we obtain:
$$F_m = \sum_{n = 0}^m (-1)^nF_n.$$

* At example 4.4, Petkovšek takes the P-finite first order recurrence $L = E - (x+1)$. After applying the substitution we get:

In [13]:
example4_4 = E - (x+1); B.recurrence(example4_4)

Sn + (-n)*Sni - n

In this case we have a recurrence with the inverse shift, which is equivalent to the recurrence $S_n^2 - n S_n - n$. This recurrence equation has as solution $y_n = n!\left(C_1 + C_2\sum_{k=1}^n(-1)^k/k!\right)$.

However in this case for $n = 0$, the original equation (with the right shift operator) provides the equality $y_1 = 0$, so we obtain $C_1 = C_2 = C$ obtaining finally
$$y(x) = C\sum_{n \geq 0} \binom{x}{n}n!\left(1 + \sum_{k=1}^n \frac{(-1)^k}{k!}\right).$$

Now, in this case, the original recurrence gives that $y(x+1) = (x+1)y(x)$ (which is the functional equation for the factorial: $y(n) = n!y(0)$). Hence, we find the following identity setting $C=1$:
$$m! = \sum_{n = 0}^m \binom{m}{n}n!\left(1 \sum_{k=1}^n\frac{(-1)^k}{k!}\right).$$

* Finally in example 4.5, Petkovšek studies the recurrence $L = E^3 - (n^2+6n+10)E^2 + (n+2)(2n+5)E-(n+1)(n+2)$. This recurrence is not so simple anymore.

In [14]:
example4_5 = E^3 - (x^2+6*x+10)*E^2 + (x+2)*(2*x+5)*E-(x+1)*(x+2); 
B.recurrence(example4_5)

Sn^3 + (-n^2 - 6*n - 7)*Sn^2 + (-2*n^2 - 8*n - 7)*Sn - n^2 - 2*n - 1

## 3. The equivalence of Proposition 2

In Proposition 2, Petkovšek provides an equivalent condition, in the case of *polynomial factorial basis*, for an operator $L$ to be $(A,B)$-compatible. Namely, $L$ is $(A,B)$ compatible if and only if:
* **C.1.** For all $n \geq 0$, $\deg(L\cdot P_n) \leq n + B$.
* **C.2.** For all $n \geq A$, $P_{n-A} | L\cdot P_n$.

This proposition will be heavily used to proved compatibility for some operators in the _product basis_ defined below. Hence, a constructive proof is heavily desired. 

### 3.1 Defining the goals for equivalence

A first step to make the result constructive is define exactly what we need to compute. For the compatibility condition is pretty clear: we need to compute the values $A$, $B$, $m$ and $\alpha_{i,j}(n)$ for arbitrary (symbolic) $n$, $0 \leq i \leq m-1$ and $-A \leq j \leq B$. 

For the conditions **C.1** and **C.2**, we need to compute the sections $m$, the bound $B$, the bound $A$ and coefficients $c_{i,j}(n)$ for arbitrary $n$, $0 \leq i \leq m-1$ and $0 \leq j \leq A+B$ such that, for all $n = km + r$:

$$\frac{L\cdot P_{n}}{P_{n-A}} = \sum_{j=0}^{A+B}c_{i,j}(k)x^j.$$

### 3.2 Increasing basis

The main tool for performing this computation is what we will call the _increasing basis_. Using the fact that we are working with *polynomial factorial basis*, we have that for any $m > n$, $\deg(P_m/P_n) = m-n$. 
* We define the $k$*-increasing basis* for $\basis$ as $\ibasis_k(\basis) = \left\{I_{k,n}\right\}_{n\geq 0}$ where
  $$I_{k,n} = \frac{P_{k+n}}{P_k}.$$
* $\ibasis_k$ **is a polynomial factorial basis for any** $k \in \nn$.

  It is clear, as we have already mention, that $\deg(I_{k,n}) = n$. And, as $\basis$ is factorial, we also have that:
  $$\frac{I_{k,n+1}}{I_{k,n}} = \frac{P_{k+n+1}P_k}{P_{k+n}P_k} = \frac{P_{k+n+1}}{P_{k+n}} = a_{n+k+1}x + b_{n+k+1},$$
  finishing the proof. $\square$
  
### 3.3 Looking to the equivalence

The compatibility condition provides the following identity:
$$L(P_n) = \sum_{i=-A}^B \alpha_{n,i} P_{n+i},$$
so if we divide this identity by $P_{n-A}$, we obtain:
$$\sum_{j=0}^{A+B} c_{n,j}x^j = \frac{L(P_n)}{P_{n-A}} = \sum_{i=-A}^B \alpha_{n,i}I_{n-A,i+A} = \sum_{j=0}^{A+B} \alpha_{n,j-A}I_{n-A, j}.$$

This implies that the computations for changing from teh compatibility to the conditions **C.1** and **C.2** are simply a change of coordinates between the canonical basis $1,x,x^2,\ldots$ and the increasing basis at an arbitrary point $n$. The computations needs to be done carefully to carry over the details of the sections, but otherwise, these computations are straightforward.
  
### What is it implemented?

The class `FactorialBasis` provides several methods to compute the equivalence of Proposition 2.

* `increasing_polynomial(k,n)`: returns the polynomial $I_{k,n}$ for the current basis. The value for $n$ must be a fixed positive integer and $k$ may take a symbolic value. This method will take care of getting the increasing basis splitted into sections in case the basis is splitted as well.
* `increasing_basis(k)`: if possible, returns the object representing the $k$-increasing basis.
* `matrix_ItP(k,m)`: returns the matrix $M_{I\rightarrow P}$ of size $m$ for the $k$-increasing basis that changes coordinates from the increasing basis to the canonical basis.
* `matrix_PtI(k,m)`: returns the matrix $M_{P\rightarrow I}$ of size $m$ for the $k$-increasing basis that changes coordinates from the canonical basis to the increasing basis.
* `equiv_DtC(compatibility)`: assuming that the input is a tuple with $(A,B,m,\alpha_{i,j}(n))$ representing the compatibility of an operator $L$ with such that for all $n = km+r$ or the operator itself, we have that
  $$L(P_{n}) = \sum_{i=-A}^{B} \alpha_{r,i}(k)P_{n+i}.$$
  This method computes the coefficients $c_{i,j}(n)$ for the equivalent condition **C.2** (i.e., the coefficients of the polynomials $L(P_{n})/P_{n-A}$):
  $$\frac{L(P_n(x))}{P_{n-A}(x)} = \sum_{i = 0}^{A+B+1} c_{r,i}(k)x^k.$$
* `equiv_CtD(A,division)`: assuming the input is of the shape of the output of `equiv_DtC` (i.e., a tuple $(A,B,m,c_{i,j}(n))$), we compute the equivalent compatibility condition (splitted in the same number of sections as deteced in the division). The output, then, will be a tuple $(A,B,m,\alpha_{i,j}(n))$ such that, for all $n = km+r$, we have:
  $$L(P_{n}(x)) = \sum_{i=-A}^B \alpha_{r,i}(k)P_{n+i}(x).$$
* `compatible_division(L)`: this method computes generically the division $L\cdot P_n/P_{n-m}$ for some $m \geq A$. If any subclass requires further information for computing this division, we implement that particular version explicitly. The output is a tuple $(A, m, D_{i,j}(n))$ where $A$ is, as usual, the lower bound of the compatibility, $m$, the number of sections and $D_{i,j}(n)$ a function such that:
  $$\frac{L\cdot P_{km+r}}{P_{km+r-s}} = D_{r,s}(k) \in \mathbb{K}[x].$$

In [15]:
## Checking how the increasing polynomials are the divisions between elements of the basis
all(
    all(
        all(
            basis.element(i+j) 
            == 
            basis.element(i)*basis.increasing_polynomial(i,j) 
            for j in range(20)) 
        for i in range(4,20))
    for basis in [P, P11, B]
)

True

In [16]:
## Checking how we get the coefficients c_{n,i}
n = B.n(); alpha = [B.alpha(example4_3)(0,i,n) for i in range(-B.A(example4_3), B.B(example4_3)+1)]
a,b,m,c = B.equiv_DtC(B.compatibility(example4_3));
all(example4_3(QQ[x](B.element(i)))/B.element(i-B.A(example4_3)) 
    ==
    sum(
        c(0,j,i)*x^j 
        for j in range(B.A(example4_3)+B.B(example4_3)+1))
    for i in range(4,10)
   )

True

## 4. Recurrence in sections

In Proposition 8, Petkosev studies how the fact that an operator is compatible with a basis makes that all the sections sequences in that basis satisfies different recurrence relations. The prove of the result is quite technical but the only thing that is important is that, for $m > 0$, if an operator is compatible in the form
$$L(P_{km+j}) = \sum_{i=-A}^B \alpha_{k,j,i} P_{km+j+i},$$
then we can define the recurrence operators for $r,j \in \{0,1,\dots,m-1\}$:
$$L_{r,j} = \sum_{\begin{array}{c}-A\leq i \leq B\\i+j \equiv r (m)\end{array}} \alpha_{k+\frac{r-i-j}{m},j,i} S_n^{\frac{r-i-j}{m}}.$$

Then with this operators we have that, for $y(x) = \sum_{k\geq 0} \sum_{j=0}^{m-1} c_{km+j}P_{km+j}$, $L(y) = 0$ if and only if for all $r \in \{0,\dots,m-1\}$ we have:
$$\sum_{j=0}^{m-1} L_{r,j}(c_{km+j})_k = 0.$$

### 4.1 Matrix operator

If we form now $R_{\basis}^m(L) = \left(L_{r,j}\right)_{r,j=0}^{m-1}$, as an extension of the original map $R$ that we saw in previous sections, then it still have the same properties: it is a ring homomorphism.

For any $m > 0$, $L_1, L_2$ compatiblo operators with $\basis$ and $c \in \kk$:
* $R_{\basis}^m(cL_1) = cR_{\basis}^m(L_1)$.
* $R_{\basis}^m(L_1 + L_2) = R_{\basis}^m(L_1) + R_{\basis}^m(L_2)$.
* $R_{\basis}^m(L_1L_2) = R_{\basis}^m(L_1)R_{\basis}^m(L_2)$.

### What is it implemented?

From the compatibility condition that we obtain with method `compatibility`, i.e., a tuple of the form $(A,B,m,\alpha_{i,j}(n))$, we can always extend this compatibility condition to any number of sections that is divisible by $m$.

Let $p = lm$ for some $l \in \mathbb{N}$. Consider $r = r_0m + r_1$ with $r \in \{0,\ldots,p-1\}$ and $r_1\in \{0,\ldots,m-1\}$. Then we can easily write:

$$L\cdot P_{kp + r} = L\cdot P_{(kl + r_0)m + r_1}= \sum_{j=-A}^B \alpha_{r_1,j}(kl+r_0) P_{kp+r+j}.$$

Hence, we have that the new compatibility coefficients are:

$$\tilde{\alpha}_{r,j}(k) = \alpha_{r\%m,j}(kp/m + r//m).$$ 

This extension is implemented in the method ``compatibility_sections(L, p)`` and returns, if possible, the compatibility tuple $(A,B,p,\tilde{\alpha}_{i,j}(n))$.

The method ``recurrence`` (described above) accepts an optional parameter ``sections`` that fix the number of sections for getting the recurrence matrix. If given, we extend the compatibility to the given number of sections and then apply the transformation to a matrix of recurrences described before.

In [17]:
B.compatibility_sections('x', 3)

(0,
 1,
 3,
 <function PSBasis.compatibility_sections.<locals>.<lambda> at 0x7fed64fc31f0>)

In [18]:
B.compatibility_matrix('x', 3)

(
      [    3*n 3*n + 1]
      [3*n + 1 3*n + 2]
0, 1, [3*n + 2 3*n + 3]
)

In [19]:
B.recurrence('x')

n*Sni + n

In [20]:
B.recurrence('x', 3)

[    3*n       0 3*n*Sni]
[3*n + 1 3*n + 1       0]
[      0 3*n + 2 3*n + 2]

## 5. Product basis

Petkovšek works with a type of basis that is composed with products of simpler basis. Namely with binomial basis. Let $\basis_1,\dots,\basis_m$ be basis of $\kk[[x]]$, we then define the new set:
$$Q_{km+r} = \prod_{i=1}^r P_{k+1}^{(i)} \prod_{i=r+1}^mP_k^{(i)}.$$

It is shown in Theorem 2 that $\basis = \prod \basis_i$ is a factorial basis when all $\basis_i$ are factorial. This provides that $X$ is $(0,1)$-compatible with $\basis$. Moreover, he shown that if $L$ is a ring homomorphism over $\kk[x]$, then $L$ is compatible with $\basis$. We can show also that if $L$ is a derivation over $\kk[[x]]$, then $L$ is also compatible with $\basis$.

This is a clear case where the compatibilities and other operations over the Product basis must be splitted into sections. In fact, this can be done in a more generic case (see below the section describing the Sieved Basis). We know show a simple case where all the compatibilities of the factors have exactly one section (just as an illutration on how the reasoning works).

### 5.1 Extending compatibility of $X$

Extending the compatibility of $X$ is pretty simple since the multiplication of polynomials is commutative. Hence, if we pick $k \in \nn$ and $r \in \{0,\dots,m-1\}$, then we have:
$$xQ_{km+r} = \left(\prod_{i=1}^{r}P_{k+1}^{(i)}\right)(xP_{k}^{(r+1)})\left(\prod_{i=r+2}^mP_{k}^{(i)}\right),$$
then, using the fact that $xP_k^{(r+1)} = \alpha_{0,0}^{(r+1)}(k)P_k^{(r+1)} + \alpha_{0,1}^{(r+1)}(k)P_{k+1}^{(r+1)}$, we have that
$$xQ_{km+r} = \alpha_{0,0}^{(r+1)}(k)Q_{km+r} + \alpha_{0,1}^{(r+1)}(k)Q_{km+r+1},$$
so we can conclude that:
$$\tilde{\alpha}_{r,j}(k) = \alpha_{0,i}^{(r+1)}(k).$$

### 5.2 Extending compatibility of endomorphisms

Let $L$ be an endomorphism of $K[x]$, i.e., $L(PQ) = L(P)L(Q)$ for any pair of polynomials. Then we can follow the prove of compatibility on the product basis:

* Proving **C.1**: given $Q_n$, we can expand it in terms of the $j$th factor basis:
  $$Q_n = \sum_{i=0}^nq_{n,i}P_n^{(j)},$$
  and applying $L$ to this expression we get
  $$L(Q_n) = \sum_{i=0}^n q_{n,i}\sum_{k=-A_j}^{B_j} \alpha_{0,k}^{(j)}(i) P_{i+k},$$j
  and a simple scan of that last expression shows that $\deg(L(Q_n)) \leq n + B_j$. As this hold for all $j = 1,...,m$, then we can conclude $\deg(L(Q_n)) \leq n+\min(B_j)$. So $B=\min(B_j : 1 \leq j \leq m)$.
  
  ***REMARK:** This proof only uses that $L$ is compatible with all the factors of $\basis$.*
  
* Proving **C.2**: if we apply directly $L$ to the product expression of $Q_n$ for $n = km+r$, we obtain:
  $$L(Q_n) = \left(\prod_{i=1}^r \sum_{l=-A_i}^{B_i} \alpha_{0,l}^{(i)}(k+1)P_{k+1+l}^{(i)}\right) \left(\prod_{i=r+1}^{m} \sum_{l = A_i}^{B_l} \alpha_{0,l}^{(i)}(k)P_{k+l}^{(i)}\right).$$
  
  If we expand this expression and see the summands, we can see they involve some product of elements from the factor basis. In fact, the minimal index we see in each case is $P_{k-A_i}^{(i)}$. Then, using the fact that all the basis are factorial, we have that for $A = \max(A_j : 1 \leq j \leq m)$, we have:
  $$Q_{n-mA} = \prod_{i=1}^r P_{k-A+1}^{(i)} \prod_{i=r+1}^mP_{k-A}^{(i)} | L(Q_n).$$
  
This two points prove that $L$ is $(mA, B)$-compatible with the product basis.

##### How is it implemented?

In order to get a constructive approach for this prove, we need to recall what we discussed in Section 3: we need to construct the division $L(Q_n)/Q_{n-mA}$. As usual, and for later use, we will consider $n = km+r$ and each of the $r$ cases will be taken separately.

If we analyze the quotient we want to study, we realize that:
$$\frac{L \cdot Q_{km+r}}{Q_{km+r-mA}} = \prod_{i=1}^r\left(\frac{L \cdot P_{k+1}^{(i)}}{P_{k+1-A}^{(i)}}\right)\prod_{i=r+1}^m\left(\frac{L \cdot P_{k}^{(i)}}{P_{k-A}}\right).$$
Each of these quotients can be coputed explicitly using the method ``compatible_division`` for each of the factorial basis, 
obtaining then that:
$$\frac{L \cdot Q_{km+r}}{Q_{km+r-mA}} = \prod_{i=1}^r D_{0,A}^{(i)}(k+1)\prod_{i=r+1}^{m} D_{0,A}^{(i)}(k) \in \mathbb{K}[x].$$

Moreover, we can also compute the corresponding *increasing basis* from a fixed $n = km+r$. Hence we can apply the corresponding change of coordinates to get the coefficients $\alpha_{r,j}(k)$. This provides directly the compatibility tuple desired.

* When creating a Product Basis, the user can specify the names of all the endomorphism he wants to compute the compatibility matrix. See the documentation of `ProductBasis?` for further information.

### 5.3 Extending compatibility of derivations

Let $L$ be a derivation of $K[x]$, i.e., $L(PQ) = L(P)Q + PL(Q)$ for any pair of polynomials. Then we can follow the prove of compatibility on the product basis:

* Proving **C.1**: exactly the same proof as for endomorhisms (see the remark above). Then $\deg(L(Q_n)) \leq n + B$ where $B = \min(B_j : 1 \leq j \leq m)$.
  
* Proving **C.2**: if we apply directly $L$ to the product expression of $Q_n$ for $n = km+r$, we obtain a summation of several terms:
  $$L(Q_n) = \sum_{i=1}^r L(P_{k+1}^{(i)})\frac{Q_n}{P_{k+1}^{(i)}} \sum_{i=r+1}^m L(P_{k}^{(i)})\frac{Q_n}{P_{k}^{(i)}}.$$
  
  Take $A = \max(A_j : 1 \leq j \leq m)$. We will show now that $Q_{n-Am}$ divides $L(Q_n)$ analyzing the divisibility of each of the summands. Let $i \leq r$:
  $$\frac{L(P_{k+1}^{(i)}Q_n}{Q_{n-mA}P_{k+1}^{(i)}}$$
  is a polynomial because the $i$th factor of $Q_{n-mA}$ (which is $P_{k+1-A}$) divides $L(P_{k+1}^{(i)})$ by th compatibility of $L$ with $\basis_i$. The $i$th factor of $Q_n$ is cancelled by $P_{k+1}^{(i)}$ and all the other factors are cancelled by $Q_{n-mA}$ since all the other basis are factorial.
  
  We can argue similarly for any $i \in \{r+1,\ldots m\}$. Hence $Q_{n-mA}$ divides $L(Q_n)$ for all $n \in \mathbb{N}$.
  
This two points prove that $L$ is $(mA, B)$-compatible with the product basis.

##### How is it implemented?

In order to get a constructive approach for this prove, we need to recall what we discussed in Section 3: we need to construct the division $L(Q_n)/Q_{n-mA}$. As usual, and for later use, we will consider $n = km+r$ and each of the $r$ cases will be taken separately.

Following the proof we have written above, the quotient of $L(Q_n)/Q_{n-mA}$ has a very special shape. More precisely:

$$\frac{L \cdot Q_{km+r}}{Q_{(k-A)m + r}} = 
\sum_{i=1}^r \left(D_{0,A}^{(i)}(k+1)\prod_{j\neq i, j = 1}^{r} I_{k+1-A,A}^{(j)} \prod_{j = r+1}^m I_{k-A,A}^{(j)}\right) + 
\sum_{i=r+1}^m \left(D_{0,A}^{(i)}(k)\prod_{j = 1}^{r} I_{k+1-A,A}^{(j)} \prod_{j\neq i, j = r+1}^m I_{k-A,A}^{(j)}\right)$$

where $D_{i,j}(k)$ can be computed with the method ``compatible_division`` for each basis and the $I_{i,j}$ can be computed with the method ``increasing_polynomial`` for each basis. This yields a polynomial that can be transformed into a compatibility condition using the ideas of method ``equiv_CtD``.

* When creating a Product Basis, the user can specify the names of all the derivations he wants to compute the compatibility matrix. See the documentation of `ProductBasis?` for further information.



##### Example 6
In this example, Petkovšek studied several product of binomial basis and their compatibilities. Here we can see we can extract the same information automatically using our package:

In [21]:
example_6a = ProductBasis([BinomialBasis(2,0), BinomialBasis(3,0)], ends=['E'])
example_6a.compatibility_matrix('E')

(
      [                                       0                    (4/9*n - 2/3)/(n - 1) (211/36*n^2 - 187/18*n + 10/3)/(n^2 - n)                      (131/12*n - 16/3)/n                         (21/2*n - 9/2)/n                                        6                                        1]
6, 0, [           (-4/27*n^2 + 2/9*n)/(n^2 - 1)     (8/27*n^2 - 28/27*n + 8/9)/(n^2 - 1)       (20/9*n^2 - 4/3*n - 2/9)/(n^2 + n)   (131/18*n^2 - 1/3*n - 17/18)/(n^2 + n)                   (17/2*n + 7/2)/(n + 1)                        (4*n + 2)/(n + 1)                                        1]
)

In [22]:
example_6b = ProductBasis([BinomialBasis(2,-1), BinomialBasis(3,4)], ends=['E'])
example_6b.compatibility_matrix('E')

(
      [                                                    0                  (4/9*n^2 - 50/9*n + 154/9)/(n^2 - n)              (211/36*n^2 + 55/6*n + 791/36)/(n^2 - n)                                  (131/12*n - 97/12)/n                                     (21/2*n + 13/2)/n                                                     6                                                     1]
6, 0, [   (-4/27*n^3 + 2/9*n^2 + 44/3*n - 1694/27)/(n^3 - n) (8/27*n^3 - 116/27*n^2 + 508/27*n - 616/27)/(n^3 - n)                (20/9*n^2 - 188/9*n + 416/9)/(n^2 + n)               (131/18*n^2 - 13/6*n + 107/9)/(n^2 + n)                               (17/2*n - 15/2)/(n + 1)                                     (4*n + 2)/(n + 1)                                                     1]
)

In [23]:
example_6c = ProductBasis([BinomialBasis(4,0), BinomialBasis(4,0)], ends=['E'])
example_6c.compatibility_matrix('E')

(
      [                                                  1    (8*n^3 - 36*n^2 + 44*n - 12)/(n^3 - 3*n^2 + 2*n) (28*n^3 - 112*n^2 + 128*n - 36)/(n^3 - 3*n^2 + 2*n)                     (56*n^2 - 112*n + 40)/(n^2 - n)                     (70*n^2 - 126*n + 44)/(n^2 - n)                                       (56*n - 28)/n                                       (28*n - 12)/n                                                   8                                                   1]
8, 0, [                                    (n - 3)/(n + 1)                                   (8*n - 8)/(n + 1)              (28*n^3 - 56*n^2 + 16*n + 4)/(n^3 - n)              (56*n^3 - 84*n^2 + 12*n + 8)/(n^3 - n)                      (70*n^2 - 14*n - 12)/(n^2 + n)                              (56*n^2 - 8)/(n^2 + n)                                 (28*n + 12)/(n + 1)                                   (8*n + 4)/(n + 1)                                                   1]
)

## 6. The method ``DefiniteSumSolutions``

The method `DefiniteSumSolutions` presented in Petkovšek's paper can be now easily implemented using all the tools described above. This method i implemented in the subpackage ``psbasis.misc``.

In [24]:
print(DefiniteSumSolutions.__doc__)


        Petkovšek's algorithm for transforming operators into recurrence equations.
        
        This method is the complete execution for the algorithm **DefiniteSumSolutions** described in
        :arxiv:`1804.02964v1`. This methods takes an operator `L` and convert the problem
        of being solution `L \cdot y(x) = 0` to a recurrence equation assuming some hypergeometric
        terms in the expansion.
        
        The operator must be a difference operator of `\mathbb{Q}[x]<E>` where `E: x \mapsto x+1`. The operator may 
        belong to a different ring from the package *ore_algebra*, but the generator must have the 
        behavior previously described.
        
        This function does not check the nature of the generator, so using this algorithm with different 
        types of operators may lead to some inconsistent results.
        
        INPUT:

        * ``operator``: difference operator to be transformed.
        * ``input``: the coefficients of the bino

###### Example 7

In this example Petkovšek applies the method ``DefiniteSumSolutions`` to several operators obtaining diverse resutls:

In [25]:
example_7_basis = ProductBasis([B, B], ends=['E']);
example_7_basis

ProductBasis
	- Binomial basis (x) choose n
	- Binomial basis (x) choose n

In [26]:
example_7_basis.recurrence('x')

[    n n*Sni]
[n + 1     n]

In [27]:
example_7_basis.compatibility_matrix('x')

(
      [    n n + 1]
0, 1, [    n n + 1]
)

In [28]:
example_7_basis.recurrence('E')

[                  Sn + 1        (2*n + 1)/(n + 1)]
[                    2*Sn ((n + 1)/(n + 2))*Sn + 1]

###### Example 7.1

Here, Petkovšek considered the linear operator $L = (n+1)E - 2(2n+1)$ and applies its algorithm to get a recurrence equation $L'$ such that
$$L'(c_k) = 0\quad \Rightarrow\quad L\left(\sum_{k \geq 0} c_k\binom{n}{k}^2\right) = 0.$$

In [29]:
example_7_1 = (x+1)*E - 2*(2*x+1);
DefiniteSumSolutions(example_7_1, 1,1,0,0)

Sn - 1

Then we can conclude that the only solution of that form has $c_k = 1$, showing that
$$\left((n+1)E - 2(2x+1)\right)\cdot\left(\sum_{k\geq 0} \binom{n}{k}^2\right) = 0,$$
which makes complete sense, since
$$\sum_{k \geq 0} \binom{n}{k}^2 = \binom{2n}{n}$$
is annihilated by $L$.

###### Example 7.2

In the last example of the paper, Petkovšek studies the following difference operator:
    $$L = 4(2n+3)^2(4n+3)E^2 - 2(4n+5)(20n^2+50n+27)E + 9(4n+7)(n+1)^2.$$
    
Again, we are looking for a new recurrence equation $L'$ such that $L'(c_k) = 0$ implies that $L\left(\sum_{k\geq 0} c_k \binom{n}{k}^2\right) = 0$.

In [30]:
example_7_2 = 4*(2*x+3)^2*(4*x+3)*E^2 - 2*(4*x+5)*(20*x^2+50*x+27)*E + 9*(4*x+7)*(x+1)^2;
DefiniteSumSolutions(example_7_2, 1,1,0,0)

(n + 1/2)*Sn - 1/4*n - 1/4

Or, equivalently as a monic operator:
$$L' = S_n - \frac{n+1}{2(2n+1)}.$$

Using other tools we can check that $h_k = \frac{1}{\binom{2k}{k}}$ is annihilated by $L'$, so we can conclude that the solution $(y_n)_{n\geq 0} = \left(\sum_{k\geq 0}c_k\binom{n}{k}\right)_{n\geq 0}$ is the sequence
$$y_n = \sum_{k \geq 0} \frac{\binom{x}{n}^2}{\binom{2k}{k}}.$$

## 7. Extending sets to basis with Product basis

We may be interested in study recurrence with the kernel $\binom{x+n}{2n}$:
$$\binom{x+n}{2n} \longrightarrow \left\{1, \frac{1}{2} x^{2} + \frac{1}{2} x, \frac{1}{24} x^{4} + \frac{1}{12} x^{3} - \frac{1}{24} x^{2} - \frac{1}{12} x, \frac{1}{720} x^{6} + \frac{1}{240} x^{5} - \frac{1}{144} x^{4} - \frac{1}{48} x^{3} + \frac{1}{180} x^{2} + \frac{1}{60}x, \ldots\right\}.$$

We can see in this example that these elements are always polynomials of even degree. This is clearly not a polynomial basis, since all the odd degrees are missing. However, in each step, we are adding 2 new roots to the polynomials:
$$\frac{\binom{x+n+1}{2(n+1)}}{\binom{x+n}{2n}} = \frac{x^2 + x - n(n+1)}{2(2n^2+3n+1)} \in \qq(n)[x],$$
which leads, in the step $n$ to add the following two roots:
$$n,\qquad -n-1.$$

We can now consider the two factorial basis generated by the root sequences $(n)_{n \in  \nn}$ and $(-n-1)_{n\in \nn}$:
$$(n)_{n\in \nn} \longrightarrow \left\{1,\ x,\ x(x-1),\ x(x-1)(x-2),\ \ldots\right\} = \{(x)^{\underline{n}}\ :\ n\in \nn\}.$$
$$(-n-1)_{n\in \nn} \longrightarrow \left\{1,\ (x+1),\ (x+1)(x+2),\ \ldots\right\} = \{(x+1)^{\overline{n}}\ :\ n\in \nn\}.$$

These two type of basis are included in the package in the class ``FallingBasis``:

In [31]:
b1 = FallingBasis(1,0,1); x = b1[1]; b1

Falling Factorial Basis (1, x, x(x-1),...)

In [32]:
b2 = FallingBasis(1,1,-1); b2

Raising Factorial Basis (1, x + 1, x + 1(x + 1+1),...)

In example 8 of Petkovšek's paper, he looked into the binomial coefficient $\binom{x+n}{2n}$ and extended it with the odd term $\binom{x+n}{2n+1}$. We can see that the product basis of the falling and raising factorials are exatly those elements except for scalar factor:

In [33]:
b1b2 = ProductBasis([b1,b2])
def example_8_binomial2k(n):
    if(n % 2 == 0):
        return binomial(x+n//2, n)
    else:
        return binomial(x+n//2, n)
[b1b2[i]/example_8_binomial2k(i) for i in range(10)]

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In fact, we can see that the scalar factor is exactly $n!$. Hence, we can get a Factorial basis which elements are exactly the two binomial coefficients Petkovšek was interested:

In [34]:
B2 = b1b2/factorial(n)
all(B2[i]/example_8_binomial2k(i) == 1 for i in range(50))

True

Since this basis is built as a product of two Factorial basis, we have automatically a compatibility by multiplication by $x$:

In [35]:
B2.recurrence('x')

[      n 2*n*Sni]
[2*n + 1  -n - 1]

Here we have an interesting phenomena: the falling factorial basis $(x)^{\underline n}$ is not compatible with the shift 
operator $E$ but this operator is compatible with the product basis of the two basis. We can see it from Proposition 3 in Petkovšek's paper, which characterize that a factorial basis $\basis$ is compatible with $E$ if and only if, for all $k \in \nn$ the root sequence of $\basis$ the following inclusion of multisets holds:
$$[\rho_1 + 1,\ldots \rho_k + 1] \subseteq [\rho_1,\rho_2,\ldots,\rho_{k+A}],$$
and in the particular case of the basis containing the binomial coefficients, we have that for $A = 2$ this
condition holds:

In [36]:
print([B2.root_sequence()(i)+1 for i in range(10)])
print([B2.root_sequence()(i) for i in range(10+2)])
all(
    multiset_inclusion(
        [B2.root_sequence()(i)+1 for i in range(k)], 
        [B2.root_sequence()(i) for i in range(k+2)]) 
    for k in range(20)
)

[1, 0, 2, -1, 3, -2, 4, -3, 5, -4]
[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5, -6]


True

We can use this data to try and guess the compatibility cnodition for the shift $E: x \mapsto x+1$. Namely, we compute the element $P_n(x+1)$ for some values of $n$ in terms of $P_{n-A}(x), \ldots, P_n(x)$, and then guess a linear recurrence for the possible coefficients of $\alpha_{i,j}(n)$ for the compatibility condition for a fixed number of sections. Then we get a rational solution for those recurrence and we provde that guess as a possible choice.

This procedure is implemented in the method ``guess_compatibility_E`` in the package ``psbasis.misc``. 

In [37]:
guessed_compatibility = guess_compatibility_E(B2, sections=2)
guessed_compatibility

(2, 0, 2, <function guess_compatibility_E.<locals>.<lambda> at 0x7fed64f9c8b0>)

The method `check_compatibility` in package `psbasis.misc` allows us to check that the compatibility condition holds for a particular basis up to a certain bound. We can check that, up to $n = 2\cdot 100$ ($2$ being the number of sections and $100$ being the argument bound in the method), the compatibility condition holds for our basis:

In [38]:
check_compatibility(B2, guessed_compatibility, lambda p: p(x=x+1), 100)

True

In [39]:
B2.set_compatibility('E', guessed_compatibility)

#### 7.1 Generic binomial basis

As a generalization of the previous example, consider the binomial coefficient
$$B(a,b,c,m,r;n) = \binom{ax + bn + c}{mn + r}.$$
Can we repeat the computations to create a basis built as a Product basis with compatibilities with $E$ and $x$?

The first step is to understand the structure of the elements we are considering. In the previous case, we had that incrementing $n$ by one, we obtained a polynomial with $+2$ on the degree. Or, equivalently:
$$\frac{\binom{x+n+1}{2n+2}}{\binom{x+n}{2n}} \in \mathbb{K}[x]_{\leq 2}.$$

We compute the same quotient the generic case using the Symbolic ring of Sage and we analize the ouput we receive:

In [40]:
a,b,c,m,r = var('a b c m r')
exp = binomial(a*x + b*n + c, m*n + r)
fexp = (exp(n=n+1)/exp).simplify_full()
fexp

factorial((b - m)*n + a*x + c - r)*factorial(b*n + a*x + b + c)*factorial(m*n + r)/(factorial((b - m)*n + a*x + b + c - m - r)*factorial(b*n + a*x + c)*factorial(m*n + m + r))

Wee see here three factors in bot the numerator and the denominator. If we compare them pairwise, we can extract useful information:

* **First factor:** we can see with this factor that we require $b \leq m \in \mathbb{N}$ to obtain a polynomial from here:
  $$\frac{((b-m)n + ax+ c -r)!}{((b-m)n+ax+b+c-m-r)!} = \prod_{i=0}^{m-b-1} ((b-m)n + ax + c - r -i) \in \mathbb{K}[x]_{\leq m-b}$$
* **Second factor:** for this case, we see we need $b \in \mathbb{N}$ too:
  $$\frac{(bn+ax+b+c)!}{(bn + ax + c)!} = \prod_{i=1}^b (bn + ax + c + i) \in \mathbb{K}[x]_{\leq b}.$$
* **Third factor:** in this case we obtain a rational function in $n$, so it will lead to a polynomial of degree $0$ in $x$:
  $$\frac{(mn+r)!}{(mn + m + r)!} = \frac{1}{\prod_{i=1}^m (mn+r+i)}.$$
  Moreover, in order to avoid a division by zero, we need to assume that $r \geq 0$.
  
Putting everything together, we can see that for the assumptions that we imposed ($a,b,m,r \in \mathbb{N}$, $b \leq m$ and $c \in \mathbb{Z}$) the elements in the sequence $B(a,b,c,m,r;n)$ are polynomials of degree $nm$. Mimicing what we did for the first example (which corresponds to $B(1,1,0,2,0;n)$) we should be able to glue togegher $m$ simpler basis knowing the sequence of roots and the leading coefficients.

###### 7.1.1 Obtaining the roots and the basic basis

If we look again to the first two factors of the quotient $B(a,b,c,m,r;n+1)/B(a,b,c,m,r;n)$ we see two families of roots associated with this sequence:

* **From the first factor**: we obtain $m-b$ roots with the following values:
  $$\frac{(m-b)n - c + r + i}{a},\quad i=0,\ldots m-b-1.$$
  For each particular $i$ we have an increasing sequences of roots:
  $$\frac{-c+r+i}{a}, \frac{-c+r+i+(m-b)}{a}, \frac{-c+r+i+2(m-b)}{a}, \frac{-c+r+i+3(m-b)}{a}, \ldots$$
  which we can easily build using the class ``FallingBasis`` by calling ``FallingBasis(a, c-r-i, (m-b))``.
* **From the second factor**: we obtain ow $b$ roots taking the following values:
  $$\frac{-bn-c-i}{a},\quad i=1,\ldots b.$$
  Now, for each particular $i$, this is a decreasing sequence of roots:
  $$\frac{-c-i}{a}, \frac{-c-i -b}{a}, \frac{-c-i -2b}{a}, \frac{-c-i -3b}{a},\ldots$$
  which, again, can be easily built using the the class ``FallingBsis`` by calling ``FallingBasis(a, c+i, b)``.

In [41]:
def symbolic_binomial(a,b,c,m,r):
    return binomial(a*x+b*n+c, m*n+r)

def GeneralizedBinomial_basis(a,b,c,m,r):
    ## Chcking the inputs
    if(any(not el in ZZ for el in (a,b,c,m,r))):
        raise TypeError("The coefficients must always be integers")
    if(any(not el >= 0 for el in (b,r))):
        raise ValueError("The coefficients 'b', and 'r' must be non-negative integers")
    if(any(not el > 0 for el in (a,m))):
        raise ValueError("The coefficients 'a' and 'm' must be positive integers")
    if(b > m):
        raise ValueError("The coefficient 'b' must be at most 'm'")
        
    ## Basis from the first factor:
    F1 = [FallingBasis(a, c-r-i, (m-b)) for i in range(m-b)]
    ## Basis from the scond factor:
    F2 = [FallingBasis(a, c+i, -b) for i in range(1, b+1)]
    
    return ProductBasis(F1 + F2)

###### 7.1.2 Obtaining the leading coefficients

We can prove by induction that the leading coefficient of $B(a,b,c,m,r;n)$ as a polynomial in $x$ is precisely
$$\frac{a^{r+mn}{(r+mn)!}.$$

* Let $n = 0$. Then we have:
  $$B(a,b,c,m,r;0) = \binom{ax+c}{r} = \frac{\prod_{i=1}^r (ax+c+i)}{r!},$$
  so the leading coefficient is, precisely, $a^r/r!$
* Let $n \geq 0$ and assume that the leading coefficient of $B(a,b,c,m,r;n) = \frac{a^{r+mn}}{(r+mn)!}$. Using the results previously shown on this notebook, we havethat the quotient of the leading coefficients of $B(a,b,c,m,r;n+1)$ and $B(a,b,c,m,r;n)$ is:
  * *From the first factor:* $a^{m-b}$.
  * *From the second factor:* $a^b$.
  * *From the third factor:* $1/\prod_{i=1}^m(mn+r+i)$,
  so putting everything together and using the induction hypothesis we obtain that:
  $$\text{lc}(B(a,b,c,m,r;n+1)) = \frac{a^{r+mn}}{(r+mn)!}\frac{a^m}{\prod_{i=1}^m (mn+r+i)} = \frac{a^{r+m(n+1))}}{(r+m(n+1))!}.$$
  
Now, from the Falling basis that we have used to build the roots, we already have a factor $a^{mn}$ in the leading coefficient. Only the $a^r/(r+mn)!$ remains to be included. 

In [42]:
def GeneralizedBinomial_scalar(a,b,c,m,r):
    return a^r/factorial(r+n)

In [43]:
def GeneralizedBinomial_wo_compatibilities(a,b,c,m,r):
    return GeneralizedBinomial_basis(a,b,c,m,r)*GeneralizedBinomial_scalar(a,b,c,m,r)

In [44]:
aux = GeneralizedBinomial_wo_compatibilities(1,1,0,2,0)
[aux[i] == B2[i] for i in range(10)]

[True, True, True, True, True, True, True, True, True, True]

###### 7.1.3 Compatibility with $E$

So far, we know several cases where the generalized Binomial basis are compatible with the shift $E:x \mapsto x+1$. For example, all cases with $b=r=0$ and $m = 1$ (which are the basic binomial coefficients covered in the algorithm `DefiniteSumSolutions`) are compatible with the shift. Also the case with $B(1,1,0,2,0)$ is also compatible (as we guessed in this Section).

We are going to study now the general case looking into Proposition 3 of Petkovšek's paper with the root multi-inclusion. Since we have studied the added roots in these generalized Binomial basis, we can study when we have that multiset inclusion and, maybe, even compute explicitly the value for the compatibility bound $A$.

As we saw preously, we have two types of roots:

* Increasing roots: provided by the first factor. They have the form:
  $$\begin{array}{ccccc}
      \frac{-c+r+i}{a}, &\frac{-c+r+i+(m-b)}{a}, &\frac{-c+r+i+2(m-b)}{a}, &\frac{-c+r+i+3(m-b)}{a}, &\ldots\qquad i = 0,\ldots,m-b-1\\
      \downarrow_{+1} & \downarrow_{+1} & \downarrow_{+1} & \downarrow_{+1} & \\
      \frac{-c+r+i+a}{a}, &\frac{-c+r+i+a+(m-b)}{a}, &\frac{-c+r+i+a+2(m-b)}{a}, &\frac{-c+r+i+a+3(m-b)}{a}, &\ldots\qquad i = 0,\ldots,m-b-1
  \end{array}$$
  Let $\iota_{i,k} = \frac{-c+r+i+k(m-b)}{a}$. 
  The we can order the increasing roots by $t = i + k(m-b)$ (i.e., $\iota_{i,k} = \iota{t}$). I this case we have that 
  $$\iota_t + 1 = \iota_{t+a},$$
  so the roots after adding 1 will appear later. Moreover, with this notation it is clear that $\iota_{t+1} + 1 = \iota_{t+a+1}$, so *the (next root) plus one is the next (root plus one)*.
  
  In particular, if we write $a = p(m-b) + q$, then we have that $\iota_{0} + 1 = \iota_{a} = \iota_{q,p}$ and all the other roots will appear in sequence after that. We can use then the first $a$ roots to compensate in the decreasing roots.
* Decreasing roots: provided by the second factor. These are the main problematic set of roots, since we may be not be able to compensate properly all the root when adding 1. These roots have the form:
  $$\begin{array}{ccccc}
      \frac{-c-i}{a}, &\frac{-c-i -b}{a}, &\frac{-c-i -2b}{a}, &\frac{-c-i -3b}{a},&\ldots\\
      \downarrow_{+1} & \downarrow_{+1} & \downarrow_{+1} & \downarrow_{+1} & \\
      \frac{-c-i+a}{a}, &\frac{-c-i+a-b}{a}, &\frac{-c-i+a-2b}{a}, &\frac{-c-i+a-3b}{a},&\ldots\\
  \end{array}$$
  Let $\delta_{i,k} = \frac{-c-i-kb}{a}$. The we can order the decreasing roots by $t = i+kb$ (i.e., $\delta_{i,k} = \delta{t}$). I this case we have that 
  $$\delta_t + 1= \frac{-c-t}{a} + 1 = \frac{-c-t+a}{a} = \delta_{t-a},$$
  so the roots after adding 1 have already appeared. Moreover, with this notation it is clear that $\iota_{t+1} + 1 = \iota_{t-a+1}$, so *the (next root) plus one is the next (root plus one)*.
  
  In particular, there are $a$ roots that we can not match. Namely, the first $\delta_{1},\ldots,\delta_{a}$ can not be mathced and need to be covered b the increasing roots.
  
So we have $a$ roots from the increasing roots that needs to match with the first $a$ decreasing roots. We can chec the formulas for both:

* Decreasing roots plus one:
  $$\frac{-c-1+a}{a}, \frac{-c-2+a}{a}, \frac{-c-3+a}{a}, \ldots, \frac{-c}{a}.$$
* Increasing roots:
  $$\frac{-c+r}{a}, \frac{-c+r+1}{a}, \frac{-c+r+2}{a}, \ldots, \frac{-c+r+a-1}{a}.$$
  
**So we need that $r = 0$**. Otherwise the shift $E$ will not be compatible with the generalized Binomial basis. *I bet that for other $r$ we will get compatibilities with other shifts.*

Now for computing the lower bound for compatibility $A$ we need to lok to $a$. Let $a = p(m-b) + q$. Then after moving the root sequence $m$ steps forward, we have added $m-b$ increasing roots and $b$ decreasing roots.  The decreasing roots are not important here, since they will be compensated right away. We only need an $A$ big enough so all increasing roots are included already, i.e., $A = pm$.

In [45]:
def GeneralizedBinomial_decreasing_roots(a,b,c,m,r):
    r'''
        Method that returns the set of decreasing roots that has to be compensated by the increasing roots.
    '''
    return [(-c-j)/a for j in range(1,a+1)]
                
def GeneralizedBinomial_increasing_roots(a,b,c,m,r):
    r'''
        Method that returns the increasing roots that can be used to compensate the decreasing basis.
    '''
    return [(-c+r+j)/a for j in range(a)]

In [46]:
a = 3; b = 1; c = 1; m = 2; r = 0
p = a//(m-b); A = p*m
aux = GeneralizedBinomial_wo_compatibilities(a,b,c,m,r); l = 2
all(multiset_inclusion([aux.rho(i)+1 for i in range(l)],[aux.rho(i) for i in range(l+A)]) for l in range(10))

True

In [47]:
guessed_compatibility = guess_compatibility_E(aux, sections=2)
check_compatibility(aux, guessed_compatibility, lambda p: p(x=x+1))

True

In [48]:
aux.set_compatibility('E', guessed_compatibility)
aux.recurrence('E')

[Sn^3 + 5*Sn^2 + 6*Sn + 1          Sn^2 + 4*Sn + 3]
[    Sn^3 + 4*Sn^2 + 3*Sn          Sn^2 + 3*Sn + 1]

###### 7.1.4 Compatibility with other shifts

Similarly to Proposition 3 from Petkovšek's paper, we can prove the following result:

**Proposition:** let $\basis$ be a factorial basis and $E_\alpha: x \mapsto x + \alpha$. Then $E_\alpha$ is $(A,0)$-compatible with $\basis$ if and only if 
$$[\rho_0 + \alpha,\ldots \rho_k + \alpha] \subseteq [\rho_0,\ldots,\rho_{k+A}]\ \text{for }k\in \mathbb{N}.$$

We can see that the only way a shift $E_\alpha$ is compatible with $B(a,b,c,m,r)$ is that $\alpha = p/a$ for some $p \in \nn$. Hence, there is a natural shift $\tilde{E}$ that generates all the compatibilities with shifts. Namely, $\tilde{E}(x) = x + 1/a$.

In this sense, we still need that $r = 0$ in order to have compatibility. In fact, we would have a $(b+1,0)$-compatibility in $m$ sections. We can then proceed as usual with our guessing tool to get the crresponding recurrence.

The compatibility with the classical shift $E$ is then trivially obtained as a iteration of $\tilde{E}$ exactly $a$ times.

In [49]:
all(multiset_inclusion([aux.rho(i)+(1/a) for i in range(l)],[aux.rho(i) for i in range(l+b+1)]) for l in range(10))

True

In [50]:
guessed_compatibility = guess_compatibility_E(aux, shift=1/a, sections=2)
check_compatibility(aux, guessed_compatibility, lambda p: p(x=x+1/a))

True

In [52]:
aux.set_compatibility('Et', guessed_compatibility)
aux.recurrence('Et')

The operator Et is already compatible with this basis -- no changes are done


[Sn + 1      1]
[    Sn      1]

And we can check that the compatibility of $\tilde{E}^a$ is exactly the compatibility we computed already for $E$:

In [54]:
APR.<Et> = QQ[]
aux.compatibility_matrix(Et^a) == aux.compatibility_matrix('E')

True

###### 7.1.5 Final code for generic binomial basis

Here we can see the complete method to create these generalized binomial basis:

In [65]:
def GeneralizedBinomial(a,b,c,m,r):
    r'''
        Method to get a basis which includes the general binomial coefficients.
        
        The binomial coefficients of shape
        
        .. MATH::
        
            \binom{ax+bn+c}{mn+r}
            
        can be (as a sequence of `n`) polynomials when `c \in \mathbb{Z}`, `a,b,m,r \in \mathbb{N}` with `a, m > 0`
        and `b \leq m`. However these polynomials have degree `nm`, so they do not form a basis of \mathbb{K}[[x]]. 
        This method creates a factorial basis as a :class:`ProductBasis` that contains the specified binomial coefficients
        in the corresponding positions of the sequence. The intermediate steps are a possible extension to obtain
        every `m` steps all the necessary roots.
        
        Moreover, when `r=0`, this basis are naturally `(b+1, 0)`-compatible with the shift `\tilde{E}: x\mapsto x+(1/a)`.
        This method includes that compatibility in the basis after computing it with a guessing procedure (see 
        :func:`guess_compatibility_E` for further information). Then we also include the compatibility with the classical
        shift `E: x\mapsto x+1` using the compatibility with `\tilde{E}`.
    '''
    ## Chcking the inputs
    if(any(not el in ZZ for el in (a,b,c,m,r))):
        raise TypeError("The coefficients must always be integers")
    if(any(not el >= 0 for el in (b,r))):
        raise ValueError("The coefficients 'b', and 'r' must be non-negative integers")
    if(any(not el > 0 for el in (a,m))):
        raise ValueError("The coefficients 'a' and 'm' must be positive integers")
    if(b > m):
        raise ValueError("The coefficient 'b' must be at most 'm'")
        
    ## Special case of binomial basis
    # TODO: check whether we can add the `r!=0` to the original BinomialBasis
    # TODO: check whether we can add the `b!=0` to the original BinomialBasis
    if(m == 1 and b == 0 and r == 0):
        return BinomialBasis(a,c) 
    
    n = PSBasis.n(None)
    
    ## Basis for the roots on the first factor:
    F1 = [FallingBasis(a, c-r-i, (m-b)) for i in range(m-b)]
    ## Basis for the roots on the second factor:
    F2 = [FallingBasis(a, c+i, -b) for i in range(1, b+1)]
    
    basis = ProductBasis(F1 + F2).scalar(a^r/factorial(r+n))
    x = basis[1].parent().gens()[0]
    
    if(r == 0):
        guessed_compatibility = guess_compatibility_E(basis, shift=1/a, sections=m)
        assert(check_compatibility(basis, guessed_compatibility, lambda p: p(x=x+1/a)))
        basis.set_compatibility('Et', guessed_compatibility)
        APR = PolynomialRing(QQ, 'Et'); Et = APR.gens()[0]
        basis.set_compatibility('E', basis.compatibility(Et^a))
        
    return basis

In [66]:
aux = GeneralizedBinomial(1,1,0,2,0)
print(all(aux[i] == B2[i] for i in range(20)))
print(aux.compatibility_matrix('E') == B2.compatibility_matrix('E'))
aux.compatible_operators()

True
True


dict_keys(['x', 'Et', 'E'])

## 8. Sieved basis

We are going to condier now another type of generalization of the Product basis. Remember that for a Product basis, we started with a set of basis $\basis_1,\ldots,\basis_m$ and we _interlaced_ them in an appropriate way, namely:
    $$P_{qm+r}(x) = \prod_{i=1}^{r}P_{i,q+1}(x) \prod_{i=r+1}^{m}P_{i,q}(x).$$
    
If we consider polynomial basis and we look into the root sequence, we are interlecing the $m$ sequences of roots for each of the factors of our basis. But we could create a new sequence in a very different way. For example, if we have two sequences, we could create the following root sequence:
    $$[\rho_{1,1},\rho_{1,2},\rho_{2,1},\rho_{1,3},\rho_{1,4},\rho_{2,2},\ldots],$$
i.e., we take two roots from the first basis and one root from the second basis. We could even mix them even more. In this section we are going to study this type of basis, that will allow us to build more examples automatically.

**Definition:** Let $\basis_0,\ldots,\basis_{F-1}$ be factorial basis and $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots, F-1\}^m$. We define the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ with the polynomials $Q_n(x)$ defined as follows (taking $n = km + r$):

$$Q_n(x) = \prod_{i=0}^{F-1}P_{e_i(n)}^{(i)}(x)$$,

where $e_i(n) = kS_i + s_i(r)$ where:
* $s_i(r) = \#\{j \in \{0,\ldots,r-1\ :\ \sigma_r = i\}$,
* $S_i = s_i(m)$.

This sieved basis do precisely the interlacing of root sequences that we were describing before. Namely, at step $n = km+r$ we add the next root of the basis $\basis_{\sigma_r}$. This generalizes the concept of **Product basis** since, when $m = F$ and $\mathbf{c} = (0,1,\ldots,F-1)$, we have that the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ is exactly the Product basis of the basis $\basis_0,\ldots,\basis_{F-1}$.

**Lemma:** Let $\basis_0,\ldots,\basis_{F-1}$ be factorial basis and $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots, F-1\}^m$. The $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$ is a factorial basis.

*Proof:* let $n = km + r \geq 1$ and $f = \sigma_r$. Then, by definition, we have:
$$\frac{Q_{n}(x)}{Q_{n-1}(x)} = \frac{P_{e_f(n)}^{(f)}(x)}{P_{e_f(n)-1}^{(f)}(x)} \in \mathbb{K}[x].$$

The last we need to proof that $\basis$ is a factorial basis is that the degree of the $n$-th element is exaclty $n$. This is equivalent to proof that $\sum_{i=0}^{F-1} e_i(n) = n$ for all $n$. But this is clear since:
* $\sum_{i = 0}^{F-1} S_i = m$ by definition
* $\sum_{i = 0}^{F-1} s_i(r) = r$ by definition.

Hence, putting these two pieces together we obtain for $n = km+r$:
$$\sum_{i=0}^{F-1} e_i(n) = \sum_{i=0}^{F-1}\left(kS_i + s_i(r) \right) = km+r = n.$$

**Fixed notation**: for the rest of this section, we will fix the following notation:
* $\basis_i = \{P_n^{(i)}(x)\}_n$ for $i = 0,\ldots,F-1$ are factorial basis.
* $\mathbf{c} = (\sigma_0,\ldots,\sigma_{m-1}) \in \{0,\ldots,F-1\}^m$
* $\basis$ is the $\mathbf{c}$-sieved basis of $(\basis_0,\ldots,\basis_{F-1})$.

### Extending the compatibility of multiplication by $X$

Since the basis $\basis$ is again a factorial basis, it is cler that it is $(0,1)$-compatible with the multiplication by $X$. Here we present how to compute this compatibility from the compatibility conditions of each of the factors $\basis_i$.

Let $\basis_i$ be (0,1)-compatible with $X$ in $t_i$ sections. Then we have rational functions $\alpha_{r,j}^{(i)}(k) \in \mathbb{K}(k)$ such that, for all $n = kt_i + r$:

$$xP_{n}^{(i)}(x) = \alpha_{r,0}^{(i)}(k)P_{n}^{(i)}(x) + \alpha_{r,1}^{(i)}(k)P_{n+1}^{(i)}(x).$$

Let $t = \text{lcm}(t_0, \ldots, t_{F-1})$ and $a_i = t/t_i$. Then $X$ is $(0,1)$-compatible with $\basis$ in $mt$ sections. Let $n = kmt + r$ and write $r = r_0m + r_1$. Then we have that:

$$xQ_n(x) = xQ_{(kt+r_0)m+r_1}(x) = x\prod_{i=0}^{F-1}P_{e_i(n)}^{(i)}(x) = 
\left(\prod_{i=0,\ i\neq\sigma_{r_1}}^{F-1}P_{e_i(n)}^{(i)}(x)\right)\left(xP_{e_{\sigma_{r_1}}(n)}^{(\sigma_{r_1})}(x)\right).$$

Using the facts that:
* $e_{\sigma_{r_1}}(n) = S_{\sigma_{r_1}}(kt + r_0) + s_{\sigma_{r_1}}(r_1)$,
* $e_{\sigma_{r_1}}(n+1) = e_{\sigma_{r_1}}(n) + 1$ and,
* $S_{\sigma_{r_1}}r_0 + s_{\sigma_{r_1}}(r_1) = r_2t_{\sigma_{r-1}}+ r3$,

then we can write:
$$xQ_n(x) = \left(\prod_{i=0,\ i\neq\sigma_{r_1}}^{F-1}P_{e_i(n)}^{(i)}(x)\right)\left(xP_{t_{\sigma_{r_1}}(a_{\sigma_{r_1}}k+r_2) + r_3}^{(\sigma_{r_1})}(x)\right) = 
\alpha_{r_3,0}^{(\sigma_{r_1})}(a_{\sigma_{r_1}}k+r_2)Q_n(x) + \alpha_{r_3,1}^{(\sigma_{r_1})}(a_{\sigma_{r-1}}k+r_2)Q_{n+1}(x)$$

and we have computed the compatibilty conditions for $\basis$ taking:

$$\alpha_{r,j}(n) = \alpha_{r_3,j}^{(\sigma_{r_1})}\left(a_{\sigma_{r-1}}n+r_2\right) \in \mathbb{K}(n)$$

### Extending the compatibility of endomorphisms

Let $L$ be an endomorphism, i.e., $L(PQ) = L(P)L(Q)$. Then we can compute the compatibility condition in several sections for $\basis$ if $L$ is compatible with all the basis $\basis_i$. Suppose that $L$ is $(A_i,B_i)$-compatible in $t_i$ sections with $\basis_i$.

Consider the following computable values:
* $t \in \mathbb{N}$ be the minimal number such that $t_i$ divides $tS_i$ for all $i = 0,\ldots, F-1$,
* $a_i = tS_i/t_i$,
* $A = \text{max}(\lceil\frac{A_i}{S_i}\rceil\ i = 0,\ldots, F-1)$,
* $b_i = AS_i - A_i$,
* $B = \text{min}(B_i\ i = 0,\ldots, F-1)$.

We are going to show that $L$ is $(mA, B)$-compatible in $mt$ sections with $\basis$. Let $n = kmt + r$ where
* $r = r_0m+r_1$ and
* $S_ir_0 + s_i(r_) = r_{2,i}t_i + r_{3,i}$ for $i = 0,\ldots, F-1$.

Then we have that:
$$\frac{L\cdot Q_n(x)}{Q_{n-Am}(x)} = \prod_{i=0}^{F-1}\frac{L\cdot P_{e_i(n)}^{(i)}(x)}{P_{e_i(n)-AS_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}\frac{L \cdot P_{S_i(kt + r_0) + s_i(r_1)}^{(i)}(x)}{P_{S_i(kt + r_0) + s_i(r_1) - AS_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}\frac{L \cdot P_{(a_ik + r_{2,i})t_i + r_{3,i}}^{(i)}(x)}{P_{(a_ik + r_{2,i})t_i + r_{3,i} - A_i - b_i}^{(i)}(x)} = 
\prod_{i=0}^{F-1}D_{r_{3,i},A_i+b_i}^{(i)}(a_ik+r_{2,i}),$$
where $D_{*,*}(*)$ are the division polynomials defined in Section 3 with the method ``compatible_division``. Hence we can
apply now the idea of ``equiv_CtD`` to compute the final coefficients $\alpha_{i,j}(k)$ for $\basis$ since we have expressed
the quotient $\frac{L\cdot Q_n}{Q_{n-Am}}$ for each $n = kmt + r$.

### Extending the compatibility of derivations

Let $L$ be a derivation, i.e., $L(PQ) = L(P)Q + PL(Q)$. Then we can compute the compatibility condition in several sections for $\basis$ if $L$ is compatible with all the basis $\basis_i$. Suppose that $L$ is $(A_i,B_i)$-compatible in $t_i$ sections with $\basis_i$.

Consider the following computable values:
* $t \in \mathbb{N}$ be the minimal number such that $t_i$ divides $tS_i$ for all $i = 0,\ldots, F-1$,
* $a_i = tS_i/t_i$,
* $A = \text{max}(\lceil\frac{A_i}{S_i}\rceil\ i = 0,\ldots, F-1)$,
* $b_i = AS_i - A_i$,
* $B = \text{min}(B_i\ i = 0,\ldots, F-1)$.

We are going to show that $L$ is $(mA, B)$-compatible in $mt$ sections with $\basis$. Let $n = kmt + r$ where
* $r = r_0m+r_1$ and
* $S_ir_0 + s_i(r_) = r_{2,i}t_i + r_{3,i}$ for $i = 0,\ldots, F-1$.

Then we have that:
$$\begin{array}{rl}
\displaystyle\frac{L\cdot Q_n(x)}{Q_{n-Am}(x)} = & \displaystyle\sum_{i=0}^{F-1}\left(\frac{L\cdot P_{e_{i}(n)}^{(i)}(x)\displaystyle\prod_{j=0,j\neq i}^{F-1} P_{e_{j}(n)}^{(j)}(x)}{Q_{n-mA}(x)}\right) \\
= & \displaystyle\sum_{i=0}^{F-1} \left(\frac{L\cdot P_{e_i(n)}^{(i)}(x)}{P_{e_i(n)-AS_i}^{(i)}(x)}\displaystyle\prod_{j=0,j\neq i}^{F-1}\frac{P_{e_j(n)}^{(j)}(x)}{P_{e_j(n)-AS_j}^{(j)}(x)}\right)\\
= & \displaystyle\sum_{i=0}^{F-1} \left(\frac{L\cdot P_{(a_ik+r_{2,i})t_i + r_{3,i}}^{(i)}(x)}{P_{(a_ik+r_{2,i})t_i + r_{3,i} - A_i - b_i}^{(i)}(x)}\displaystyle\prod_{j=0,j\neq i}^{F-1}\frac{P_{(a_jk+r_{2,j})t_j + r_{3,j}}^{(j)}(x)}{P_{(a_jk+r_{2,j})t_j + r_{3,j} - A_j - b_j}^{(j)}(x)}\right)\\
= & \displaystyle\sum_{i=0}^{F-1} \left(D_{r_{3,i},A_i+b_i}^{(i)}(a_ik+r_{2,i})\displaystyle\prod_{j=0,j\neq i}^{F-1} I_{(a_jk+r_{2,j})t_j + r_{3,j} - A_j - b_j, A_j+b_j}^{(j)}\right),
\end{array}$$
where $D_{*,*}(*)$ are the division polynomials defined in Section 3 with the method ``compatible_division`` and $I_{*,*}$ are the increasing polynomials defined in Section 3 with the method ``increasing_polynomial``. Hence we can apply now the idea of `equiv_CtD` to compute the final coefficients $\alpha_{i,j}(k)$ for $\basis$ since we have expressed the quotient $\frac{L\cdot Q_n}{Q_{n-Am}}$ for each $n = kmt + r$.

###### Example 8

In example 8 of Petkovšek's paper, he looked into a basis built as follows:

* $P_{3n}(x) = \binom{x}{n}\binom{x+n}{2n}$
* $P_{3n+1}(x) = \binom{x}{n}\binom{x+n}{2n+1}$
* $P_{3n+2}(x) = \binom{x}{n+1}\binom{x+n}{2n+1}$

Let $\mathcal{A}_2$ be such basis. This is a clear example of a Sieved basis. In particular, if we consider $\basis$ the binomial basis and $\basis_2$ the basis we built in Section 7 to represent the second factor, we then have that Petkovšek's basis is exactly the $(1,0,1)$-sieved basis of $(\basis,\basis_2)$.

In [42]:
def example_8_apery2(n):
    k = n//3; r = n%3
    if(r == 0): return binomial(x,k)*binomial(x+k,2*k)
    elif(r == 1): return binomial(x,k)*binomial(x+k,2*k+1)
    else: return binomial(x,k+1)*binomial(x+k,2*k+1)
A2 = SievedBasis([B,B2], [1,0,1], ends=['E'])
all(A2[i] == example_8_apery2(i) for i in range(50))

True

Since both $\basis$ and $\basis_2$ had compatibilities with $X$ and $E$, the basis $\mathcal{A}_2$ has already defined (using an automatic extension) the compatibilities for those two operators:

In [43]:
A2.compatibility_matrix('x')

(
      [      n 2*n + 1]
      [      n   n + 1]
0, 1, [ -n - 1 2*n + 2]
)

In [44]:
A2.compatibility_matrix('E')

(
      [                      1           (4*n - 3/2)/n                     3/2                       1]
      [    (n - 1/2)/(n + 1/2)         1/2*n/(n + 1/2) (3/2*n + 1/2)/(n + 1/2)                       1]
3, 0, [                      0                       1       (3*n + 2)/(n + 1)                       1]
)

And now we can go to the linear difference operator from Example 8:
$$L = (x+2)^2E^2 - (11x^2 + 3x + 25)E - (x+1)^2,$$
and use a similar approach as in the method `DefiniteSumSolutions` to get a recurrence that the sequence $(c_n)_n$ has to satisfy if we assume that $L\cdot \left(\sum_{n\geq 0}c_n \binom{x}{n}\binom{x+n}{2n}\right) = 0$:

In [45]:
L = (x+2)^2*E^2 - (11*x^2 + 33*x + 25)*E - (x+1)^2
recurrence_matrix = A2.recurrence(L)
first_column = [A2.remove_Sni(recurrence_matrix.coefficient((j,0))) for j in range(recurrence_matrix.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n + 1)*Sn - 4*n - 2

Here, the sequence $c_n = \binom{2n}{n}$ is a solution to that recurrence equation. Using the equality
$$\binom{2n}{n}\binom{x}{n}\binom{x+n}{2n} = \binom{x}{n}^2\binom{x+n}{n},$$
we can conclude that the fucntion
$$y(x) = \sum_{n \geq 0} c_n \binom{x}{n}\binom{x+n}{2n} = \sum_{n\geq 0} \binom{x}{n}^2\binom{x+n}{n},$$
is a solution to $L\cdot y = 0$ and, in particular, the sequence $y_k = y(k)$ (which is the Apéry's $\zeta(2)$-sequence) satisfies the recurrence:
$$(k+2)^2 y_{k+2} - (11k^2 + 33^k+25)y_{k+1} - (k+1)^2y_k = 0.$$

###### Example 9

Finally in example 9 of Petkovšek's paper, we consider a new basis with the following format:

* $P_{4n}(x) = \binom{x+n}{2n}^2$
* $P_{4n+1}(x) = \binom{x+n}{2n}\binom{x+n}{2n+1}$
* $P_{4n+2}(x) = \binom{x+n}{2n+1}^2$
* $P_{4n+3}(x) = \binom{x+n}{2n+1}\binom{x+n+1}{2n+2}$

Let $\mathcal{A}_3$ be this basis. In this case, we have a Product basis of the basis $\basis_2$ (`B2`in the code) with itself

In [46]:
def example_9_apery3(n):
    k = n//4; r = n%4
    if(r == 0): return binomial(x+k,2*k)^2
    elif(r == 1): return binomial(x+k,2*k)*binomial(x+k,2*k+1)
    elif(r == 2): return binomial(x+k,2*k+1)^2
    else: return binomial(x+k,2*k+1)*binomial(x+k+1,2*k+2)
A3 = ProductBasis([B2,B2], ends=['E'])
all(A3[i] == example_9_apery3(i) for i in range(50))

True

Since $\basis_2$ had compatibilities with $X$ and $E$, the basis $\mathcal{A}_3$ has already defined (using an automatic extension) the compatibilities for those two operators:

In [47]:
A3.compatibility_matrix('x')

(
      [      n 2*n + 1]
      [      n 2*n + 1]
      [ -n - 1 2*n + 2]
0, 1, [ -n - 1 2*n + 2]
)

In [48]:
A3.compatibility_matrix('E')

(
      [                    1           (4*n - 1)/n           (3*n - 1)/n                     2                     1]
      [  (n - 1/2)/(n + 1/2)   (n - 1/2)/(n + 1/2)           n/(n + 1/2) (2*n + 1/2)/(n + 1/2)                     1]
      [                    0                     0                     1                     2                     1]
4, 0, [                    0                     1   (3*n + 5/2)/(n + 1)   (2*n + 3/2)/(n + 1)                     1]
)

And now we can go to the linear difference operator from Example 8:
$$L = (x+2)^3E^2 - (2x+3)(17x^2+51x+39)E + (x+1)^3,$$
and use a similar approach as in the method `DefiniteSumSolutions` to get a recurrence that the sequence $(c_n)_n$ has to satisfy if we assume that $L\cdot \left(\sum_{n\geq 0}c_n \binom{x+n}{2n}^2\right) = 0$:

In [49]:
L = (x+2)^3*E^2 - (2*x+3)*(17*x^2+51*x+39)*E + (x+1)^3
recurrence_matrix = A3.recurrence(L)
first_column = [A3.remove_Sni(recurrence_matrix.coefficient((j,0))) for j in range(recurrence_matrix.nrows())]
gcrd = first_column[0].gcrd(*first_column[1:])
gcrd

(n^2 + 2*n + 1)*Sn - 16*n^2 - 16*n - 4

In this case the sequence $c_n = \binom{2n}{n}^2$ satisfies this recurrence equation. Using the same equality as in example 8, we obtain now that :
$$y(x) = \sum_{n\geq 0} c_n \binom{x+n}{2n}^2 = \sum_{n\geq 0} \binom{x}{n}^2\binom{x+n}{n}^2,$$
is a solution to $L\cdot y = 0$ and, in particular, the sequence $y_k = y(k)$ (which is the Apéry's $\zeta(3)$-sequence) satisfies the recurrence:
$$(k+2)^3y_{k+2} - (2k+3)(17k^2+51k+39)y_{k+1} + (k+1)^3y_{k} = 0.$$

## 9. Relation with composition (NOT REVIEWED)

Let $f(x) \in K[[x]]$ with $f(0) = 0$ and $f'(0) \neq 0$. Then it is easy to check that the set $\{1, f(x), f(x)^2,\ldots\}$ is an order basis of $K[[x]]$. Then for any compatible operator $L$ we can find a holonomic $\tilde{L}$ recurrence operator such that if $g(x) = \sum c_n x^n$ and $\tilde{L} \cdot (c_n) = 0$, then any solution of $L\cdot y(x) = 0$ is of the form $y(x) = g(f(x))$.

Then the question arises naturally: given $f(x)$ a D-finite function, which operators are compatible with its corresponding basis.

#### 9.1. The exponential function

Let start with a simple function: the exponential $e^x$. The first thing to look at is that the exponential does not satifies the conditions on initial values to be suitable of forming a basis of $K[[x]]$. That is not a problem: we only consider $f(x) = (e^x -1)$.

Let $f_n(x) = (e^x-1)^n$, we have clearly that $\{ f_n\ :\ n \in \nn\}$ is a basis of $K[[x]]$. The compatibilities we may find easily are:
* $f_1 f_n = f_{n+1}$.
* $e^x f_n = (f_1  + 1)f_n = f_{n+1} + f_n$.
* $\partial f_n = n e^x f_{n-1} = nf_n + n f_{n+1}$.

**Proposition:** any linear differential operator generated by $e^x$ and $\partial$ can be expressed as a composition of a D-finite function and the exponential.

###### What is it implemented?

In the package ``psbasis`` the user may find a subpackage called ``func_basis`` that contains the code for expressing the compatibilities with the exponential basis.

In fact, the object ``ExponentialBasis`` has all the previous information available so we can start experimenting with this basis.


In [23]:
EB = ExponentialBasis(); 
oE = EB.recurrence('E'); oD = EB.recurrence('Dx');

**Equation $(\partial - e^x)\cdot y(x) = 0$.**

Assume we know that the initial conditions of $y(x)$ are $1,1,2,5,15$. Since the differential equation only involves $\partial$ and $e^x$, then $y(x) = g(e^x-1)$. 

Using the compatibility conditions we conclude that the generating sequence of $g(x)$ satisfies the following recurrence:

In [24]:
rec_g = oD - oE; rec_g

(n + 1)*Sn - Sni + n - 1

Using the initial conditions we have directly that the initial conditions for $g(x)$ are $1,1,1,1,1$, and using the equivalence between holonomic recurrence and holonomic functions, we can compute the D-finite differential equation for $g(x)$:

In [25]:
def SnSni_toDx(psbasis, operator, toDx):
    R = operator.parent(); Sn = R('Sn'); Sni = R('Sni'); n = R.base()('n');
    S = OreAlgebra(R.base(), ('Sn', lambda p : p(n=n+1), lambda p : 0));
    return S(str(psbasis.reduce_SnSni((Sn**operator.degree(Sni))*operator))).to_D(toDx);

def build_DFinite(psbasis, operator, initial):    
    RDx = OreAlgebra(PolynomialRing(QQ, 'x'), ('Dx', lambda p : p, lambda p : p.derivative()));
    return DFinite.element(SnSni_toDx(psbasis,operator, RDx), initial);

In [26]:
g = build_DFinite(EB, rec_g, [1,1,1,1]); g

(3:3:8)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

Then $y(x)$ was DD-finite (we knew from the differential equation) is the composition of two D-finite functions.

**Equation $(\partial^2 - e^x\partial + e^{2x})\cdot y(x) = 0$.**

For this example we are going to use the ``sequences`` package, developed in parallel to get the initial conditions of $g(x)$ automatically. 

In [29]:
from sequences import *;
y = DDFinite.element([Exp(2*x), - Exp(x), 1], [1,0]);
rec_g = oD^2 - oE*oD + oE^2;
g = build_DFinite(EB, rec_g, []);
seq_g = inverse_faa_f(DDSequence(Exp(x)-1, True), DDSequence(y, True));
g = g.change_init_values([seq_g[i] for i in range(4)]);
g

(6:6:24)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field))

In [30]:
g(Exp(x)-1) == y

True